In [1]:
import quant_inf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

dict_color = {
    'quantum':'#1f77b4',
    'quant_diag':'#ff7f0e',
    'quant_greedy': '#2ca02c',
    'logdet':'#d62728',
    'TRW':'#9467bd',
    'quant_exp':'#8c564b',
    'C1': '#e377c2',
    'C2': '#7f7f7f',
    'C3': '#bcbd22',
    'C4': '#17becf'}

In [2]:
sns.set_context("notebook", font_scale=1)

# Coefficient from TRW article

In [1]:
def get_result_experiment_1(
        d:int,
        max_features_greedy:int,
        n_sample:int,n_points:int,
        max_coupling_strenght:float,
        interaction:str):
    """Compare different relaxation with random coefficients from [1]

    Returns:
        pandas.core.frame.DataFrame: a dataframe with columns
        ['coupling_strenght','relaxation','logp_bound','true_logp','l1_error','d']
    
    Args:
        d (int): number of variables
        max_features_greedy (int): number of features selected for greedy algorithm
        n_sample (int): number of sample for each coupling strenght
        n_points (int): number of point on the x line
        max_coupling_strenght (float): maximal value for coupling strenght
        interaction (str): type of interaction
    
    Reference:
    [1] M.J. Wainwright, T.S. Jaakkola, and A.S. Willsky. “A New Class of
        Upper Bounds on the Log Partition Function”. In: IEEE Transactions
        on Information Theory 51.7 (July 2005), pp. 2313–2335.
    """
    assert interaction in ["mixed","attractive"]
    results = pd.DataFrame(columns=['coupling_strenght','relaxation','logp_bound','true_logp','l1_error','d'])

    list_w = np.linspace(start=0,stop=max_coupling_strenght,num=n_points,endpoint=True)
    
    features_1 = (
    [set()] 
    + [{i} for i in range(1,d+1)]
    )

    for w in tqdm(list_w):
        for _ in range(n_sample):

            coefficients = quant_inf.tools.ramdom_coefficient_TRW(
                d=d,
                strenght=w,
                graph="complete",
                interaction=interaction
                )
            
            exact_inference = quant_inf.algorithms.ExactBruteForce(
                coefficients=coefficients,
                d=d,
                features=features_1)
            exact_inference.solve()

            TRW_inference = quant_inf.algorithms.TRWRelaxation(
                coefficients=coefficients,
                d=d)
            TRW_inference.solve()
            results.loc[len(results)] = [
                w,
                "TRW",
                TRW_inference.log_partition,
                exact_inference.log_partition,
                TRW_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            quant_inference = quant_inf.algorithms.QuantumRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1)
            quant_inference.solve()
            results.loc[len(results)] = [
                w,
                "quantum",
                quant_inference.log_partition,
                exact_inference.log_partition,
                quant_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            quant_greedy_inference = quant_inf.algorithms.QuantGreedyRelaxation(
                coefficients=coefficients,
                d=d)
            quant_greedy_inference.solve(number_extra_features=max_features_greedy)
            results.loc[len(results)] = [
                w,
                "quant_greedy",
                quant_greedy_inference.log_partition,
                exact_inference.log_partition,
                quant_greedy_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            logdet_inference = quant_inf.algorithms.LogDetRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1)
            logdet_inference.solve()
            results.loc[len(results)] = [
                w,
                "logdet",
                logdet_inference.log_partition,
                exact_inference.log_partition,
                logdet_inference.l1_error(exact_inference.marginals),
                d
                ]
    return results

def plot_experiment_1(result_experiment,title:str="",file:str=""):
    fig = plt.figure(figsize=(5,4.5))
    
    to_plot = result_experiment.assign(normalized_error_in_bound=lambda x: (x.logp_bound - x.true_logp)/x.d)
    ax1 = plt.subplot(3,1,(1,2))
    sns.lineplot(
        data=to_plot,
        x="coupling_strenght",
        y="normalized_error_in_bound",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax1
    )
    ax1.set_ylabel("Normalized error in bound")
    ax1.set_xlabel(None)
    plt.setp(ax1.get_xticklabels(), visible=False)
    ax1.axhline(y=0,color="grey")

    ax2 = plt.subplot(3,1,3,sharex=ax1)
    sns.lineplot(
        data=to_plot,
        x="coupling_strenght",
        y="l1_error",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax2,
        legend=False
    )
    ax2.set_ylabel("Error in\n marginals")
    ax2.set_xlabel("Coupling strenght")
    
    fig.suptitle(title)
    fig.tight_layout()
    if file != "":
        plt.savefig(file)
    plt.show()

## Running the experiment

In [ ]:
d=5
result_experiment_1_attractive= get_result_experiment_1(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    max_coupling_strenght=.5,
    interaction="attractive"
    )
result_experiment_1_mixed= get_result_experiment_1(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    max_coupling_strenght=.5,
    interaction="mixed"
    )

## Plots

In [ ]:
print("Full with attractive coupling")
plot_experiment_1(result_experiment_1_attractive)

In [ ]:
print("Full with mixed coupling")
plot_experiment_1(result_experiment_1_mixed)

# Coefficients from logdet article

In [12]:
def get_result_experiment_2(
        d:int,
        max_features_greedy:int,
        n_sample:int,n_points:int,
        max_coupling_strenght:float,
        interaction:str):
    """Compare different relaxation with random coefficients from [1]

    Returns:
        pandas.core.frame.DataFrame: a dataframe with columns
        ['coupling_strenght','relaxation','logp_bound','true_logp','l1_error','d']
    
    Args:
        d (int): number of variables
        max_features_greedy (int): number of features selected for greedy algorithm
        n_sample (int): number of sample for each coupling strenght
        n_points (int): number of point on the x line
        max_coupling_strenght (float): maximal value for coupling strenght
        interaction (str): type of interaction
    
    Reference:
    [1] Michael Jordan and Martin J Wainwright. “Semidefinite Relaxations
        for Approximate Inference on Graphs with Cycles”. In: Advances in
        Neural Information Processing Systems. Vol. 16. MIT Press, 2003.
    """
    assert interaction in ["mixed","attractive","repulsive"]
    results = pd.DataFrame(columns=['coupling_strenght','relaxation','logp_bound','true_logp','l1_error','d'])

    list_w = np.linspace(start=0,stop=max_coupling_strenght,num=n_points,endpoint=True)
    
    features_1 = (
    [set()] 
    + [{i} for i in range(1,d+1)]
    )

    for w in tqdm(list_w):
        for _ in range(n_sample):

            coefficients = quant_inf.tools.ramdom_coefficient_logdet(
                d=d,
                strenght=w,
                graph="complete",
                interaction=interaction
                )
            
            exact_inference = quant_inf.algorithms.ExactBruteForce(
                coefficients=coefficients,
                d=d,
                features=features_1)
            exact_inference.solve()

            TRW_inference = quant_inf.algorithms.TRWRelaxation(
                coefficients=coefficients,
                d=d)
            TRW_inference.solve()
            results.loc[len(results)] = [
                w,
                "TRW",
                TRW_inference.log_partition,
                exact_inference.log_partition,
                TRW_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            quant_inference = quant_inf.algorithms.QuantumRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1)
            quant_inference.solve()
            results.loc[len(results)] = [
                w,
                "quantum",
                quant_inference.log_partition,
                exact_inference.log_partition,
                quant_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            quant_greedy_inference = quant_inf.algorithms.QuantGreedyRelaxation(
                coefficients=coefficients,
                d=d)
            quant_greedy_inference.solve(number_extra_features=max_features_greedy)
            results.loc[len(results)] = [
                w,
                "quant_greedy",
                quant_greedy_inference.log_partition,
                exact_inference.log_partition,
                quant_greedy_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            logdet_inference = quant_inf.algorithms.LogDetRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1)
            logdet_inference.solve()
            results.loc[len(results)] = [
                w,
                "logdet",
                logdet_inference.log_partition,
                exact_inference.log_partition,
                logdet_inference.l1_error(exact_inference.marginals),
                d
                ]
    return results

## Running the experiment

In [ ]:
d=5
result_experiment_2_attractive= get_result_experiment_2(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    max_coupling_strenght=.5,
    interaction="attractive"
    )
result_experiment_2_mixed= get_result_experiment_2(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    max_coupling_strenght=.5,
    interaction="mixed"
    )
result_experiment_2_repulsive= get_result_experiment_2(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    max_coupling_strenght=.5,
    interaction="repulsive"
    )

## Plots

In [ ]:
print("Full with attractive coupling")
plot_experiment_1(result_experiment_2_attractive)

In [ ]:
print("Full with mixed coupling")
plot_experiment_1(result_experiment_2_mixed)

In [ ]:
print("Full with repulsive coupling")
plot_experiment_1(result_experiment_2_repulsive)

# Temperature scaling

In [18]:
def get_result_experiment_3(
        d:int,
        max_features_greedy:int,
        n_sample:int,n_points:int,
        min_temperature:float,
        max_temperature:float
        ):
    """Compare different relaxation with random normal coefficients 
    and temperature scaling

    Returns:
        pandas.core.frame.DataFrame: a dataframe with columns
        ['coupling_strenght','relaxation','logp_bound','true_logp','l1_error','d']
    
    Args:
        d (int): number of variables
        max_features_greedy (int): number of features selected for greedy algorithm
        n_sample (int): number of sample for each coupling strenght
        n_points (int): number of point on the x line
        max_temperature (float): maximal value for temperature
    """
    results = pd.DataFrame(columns=['temperature','relaxation','logp_bound','true_logp','l1_error','d'])

    list_eps = np.linspace(start=min_temperature,stop=max_temperature,num=n_points,endpoint=True)
    features_1 = (
    [set()] 
    + [{i} for i in range(1,d+1)]
    )
    complete_graph_features = (
        [{i} for i in range(1,d+1)]
        + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
    )

    rho_fixed = 2*(np.ones((d,d)) - np.eye(d))/d

    for _ in tqdm(range(n_sample)):
        coefficients = quant_inf.tools.random_coefficients_gaussian(
                graph_features=complete_graph_features
                )
        for eps in list_eps:
            exact_inference = quant_inf.algorithms.ExactBruteForce(
                coefficients=coefficients,
                d=d,
                features=features_1,
                eps=eps)
            exact_inference.solve()

            TRW_inference = quant_inf.algorithms.TRWRelaxation(
                coefficients=coefficients,
                d=d,
                eps=eps)
            TRW_inference.solve_rho_fixed(rho=rho_fixed)
            results.loc[len(results)] = [
                eps,
                "TRW",
                TRW_inference.log_partition,
                exact_inference.log_partition,
                TRW_inference.l1_error(exact_inference.marginals),
                d
                ]

            quant_inference = quant_inf.algorithms.QuantumRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1,
                eps=eps)
            quant_inference.solve()
            results.loc[len(results)] = [
                eps,
                "quantum",
                quant_inference.log_partition,
                exact_inference.log_partition,
                quant_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            quant_greedy_inference = quant_inf.algorithms.QuantGreedyRelaxation(
                coefficients=coefficients,
                d=d,
                eps=eps)
            quant_greedy_inference.solve(number_extra_features=max_features_greedy,tol=1e-2,max_iter=100000)
            results.loc[len(results)] = [
                eps,
                "quant_greedy",
                quant_greedy_inference.log_partition,
                exact_inference.log_partition,
                quant_greedy_inference.l1_error(exact_inference.marginals),
                d
                ]
            
            logdet_inference = quant_inf.algorithms.LogDetRelaxation(
                coefficients=coefficients,
                d=d,
                features=features_1,
                eps=eps)
            logdet_inference.solve()
            results.loc[len(results)] = [
                eps,
                "logdet",
                logdet_inference.log_partition,
                exact_inference.log_partition,
                logdet_inference.l1_error(exact_inference.marginals),
                d
                ]
    return results

def plot_experiment_3(result_experiment,title:str="",file:str=""):
    fig = plt.figure(figsize=(5,4.5))
    
    to_plot = result_experiment.assign(normalized_error_in_bound=lambda x: (x.logp_bound - x.true_logp)/x.d)
    ax1 = plt.subplot(3,1,(1,2))
    sns.lineplot(
        data=to_plot,
        x="temperature",
        y="normalized_error_in_bound",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax1
    )
    ax1.set_ylabel("Normalized error in bound")
    ax1.set_xlabel(None)
    plt.setp(ax1.get_xticklabels(), visible=False)
    ax1.axhline(y=0,color="grey")
    
    ax2 = plt.subplot(3,1,3,sharex=ax1)
    sns.lineplot(
        data=to_plot,
        x="temperature",
        y="l1_error",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax2,
        legend=False
    )
    ax2.set_ylabel("Error in\n marginals")
    ax2.set_xlabel("Temperature")
    
    ax1.legend(loc = 'best')

    fig.suptitle(title)
    fig.tight_layout()
    if file != "":
        plt.savefig(file)
    plt.show()

In [ ]:
d=10
result_experiment_3= get_result_experiment_3(
    d=d,
    max_features_greedy=3,
    n_sample=10,
    n_points=10,
    min_temperature=.1,
    max_temperature=10,
    )

## Plots

In [ ]:
print("Temperature scaling")
plot_experiment_3(result_experiment_3)